First recommendation engine is based on user-based collaborative filtering: In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

In [19]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv("./data/books.csv")
ratings = pd.read_csv("./data/ratings.csv")

In [3]:
ratings[:5]

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [5]:
# create a user item matrix
user_book_matrix = ratings.groupby(['user_id', 'book_id'])["rating"].max().unstack()

In [6]:
user_book_matrix

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,4.0,4.0,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4.0,4.0,4.0,NaN,5.0,NaN,NaN,NaN,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def get_read_books(user_book_matrix, user_id):
    """TODO"""
    read_books = list(user_book_matrix.loc[user_id][~user_book_matrix.loc[user_id].isna()].keys())
    return read_books
    
def create_user_book_dict(user_book_matrix):
    """TODO"""
    all_users = user_book_matrix.index.to_numpy()
    books_read = {}
    for user in all_users:
        books_read[user] = get_read_books(user_book_matrix, user)
        
    return books_read

In [24]:
books_read = create_user_book_dict(user_book_matrix)

In [46]:
def compute_euclidean_dist(user1, user2):
    """TODO"""
    books_user1 = books_read[user1]
    books_user2 = books_read[user2]
    both_read = list(set(books_user1).intersection(books_user2))
    if len(both_read) > 1:
        a = user_book_matrix.loc[user1][both_read]
        b = user_book_matrix.loc[user2][both_read]

        dist = np.linalg.norm(a-b)
    else:
        dist = 10000*5 # highest possible difference
    return dist

In [48]:
def calculate_dist_matrix():
    all_users = user_book_matrix.index.to_numpy()
    user1 = []
    user2 = []
    dist = []
    for user_id1 in all_users:
        for user_id2 in all_users:
            d = compute_euclidean_dist(user_id1, user_id2)
            user1.append(user_id1)
            user2.append(user_id2)
            dist.append(d)
    
    dist_df = pd.DataFrame({"user1": user1, "user2": user2, "dist": dist})
    return dist_df
        

In [49]:
dist_df = calculate_dist_matrix()

KeyboardInterrupt: 

how to consider the number of books they have in common?

In [45]:
user_book_matrix.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            53415, 53416, 53417, 53418, 53419, 53420, 53421, 53422, 53423,
            53424],
           dtype='int64', name='user_id', length=53424)